<a href="https://colab.research.google.com/github/Thompaiva/bird_project/blob/main/Projeto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten, Activation, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
from tensorflow import keras
from keras.layers import Conv2D, Dropout, Flatten, Dense, MaxPooling2D, Activation, BatchNormalization, AveragePooling2D, Add, ReLU

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import zipfile

!mkdir "data"
with zipfile.ZipFile('/content/drive/MyDrive/iron hack final project/archive (1).zip', 'r') as zip_ref:
    zip_ref.extractall("data")

In [ ]:
batch_size = 32
img_height = 224
img_width = 224
image_size = (224, 224, 3)
num_classes = 525
data_dir = "/content/data/train"

In [ ]:
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="both",
    seed=42,
    image_size=(299, 299),
    batch_size=batch_size,
)

Found 84635 files belonging to 525 classes.
Using 67708 files for training.
Using 16927 files for validation.


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE).cache()
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE).cache()

# MODELO SIMPLES

In [ ]:
model = Sequential()


model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

# Adicione camadas totalmente conectadas
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 128)      0

In [ ]:
model.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50
2116/2116 [==============================] - 34s 15ms/step - loss: 6.3394 - accuracy: 0.0032 - val_loss: 6.2615 - val_accuracy: 0.0027
Epoch 2/50
2116/2116 [==============================] - 31s 15ms/step - loss: 6.2360 - accuracy: 0.0045 - val_loss: 6.2597 - val_accuracy: 0.0030
Epoch 3/50
2116/2116 [==============================] - 31s 15ms/step - loss: 5.9765 - accuracy: 0.0139 - val_loss: 5.6298 - val_accuracy: 0.0334
Epoch 4/50
2116/2116 [==============================] - 31s 15ms/step - loss: 5.3922 - accuracy: 0.0571 - val_loss: 5.1501 - val_accuracy: 0.0804
Epoch 5/50
2116/2116 [==============================] - 31s 15ms/step - loss: 4.6658 - accuracy: 0.1331 - val_loss: 4.6399 - val_accuracy: 0.1501
Epoch 6/50
2116/2116 [==============================] - 31s 15ms/step - loss: 3.9343 - accuracy: 0.2267 - val_loss: 4.6718 - val_accuracy: 0.1795
Epoch 7/50
2116/2116 [==============================] - 31s 15ms/step - loss: 3.3126 - accuracy: 0.3236 - val_loss: 4.8521 -

KeyboardInterrupt: ignored

# 1 RESNET

In [ ]:


input = keras.layers.Input(shape=(img_height, img_width, 3))
rescale = keras.layers.Rescaling(1.0 / 255)(input)
conv_in = keras.layers.Conv2D(
    filters=64, kernel_size=(7, 7), strides=(2, 2), activation="relu"
)(rescale)
pool_in = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding = "same")(conv_in)
conv_r1_1 = keras.layers.Conv2D(64, 3, 1, padding = "same")(pool_in)
bn_r1_1 = keras.layers.BatchNormalization()(conv_r1_1)
relu_r1_1 = keras.layers.ReLU()(bn_r1_1)
conv_r1_2 = keras.layers.Conv2D(64, 3, 1, padding = "same")(relu_r1_1)
bn_r1_2 = keras.layers.BatchNormalization()(conv_r1_2)
skip_r1 = keras.layers.Add()([bn_r1_2, pool_in])
relu_r1_2 = keras.layers.ReLU()(skip_r1)
pool_out = keras.layers.GlobalAveragePooling2D()(relu_r1_2)
dense_out = keras.layers.Dense(512, activation="relu")(pool_out)
output = keras.layers.Dense(num_classes, activation="softmax")(dense_out)

resnet18 = keras.Model(inputs=input, outputs=output)







resnet18.compile(optimizer=keras.optimizers.SGD(learning_rate=1), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
resnet18.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv2d_4 (Conv2D)              (None, 109, 109, 64  9472        ['rescaling[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
r = resnet18.fit(train_ds, validation_data=val_ds, epochs = 50, batch_size = batch_size)

Epoch 1/50
2116/2116 [==============================] - 29s 13ms/step - loss: 5.6367 - accuracy: 0.0208 - val_loss: 6.0281 - val_accuracy: 0.0275
Epoch 2/50
2116/2116 [==============================] - 27s 13ms/step - loss: 4.7970 - accuracy: 0.0719 - val_loss: 5.8025 - val_accuracy: 0.0584
Epoch 3/50
2116/2116 [==============================] - 27s 13ms/step - loss: 4.2406 - accuracy: 0.1353 - val_loss: 5.2665 - val_accuracy: 0.0810
Epoch 4/50
2116/2116 [==============================] - 27s 13ms/step - loss: 3.8419 - accuracy: 0.1904 - val_loss: 3.8761 - val_accuracy: 0.1916
Epoch 5/50
2116/2116 [==============================] - 27s 13ms/step - loss: 3.5205 - accuracy: 0.2382 - val_loss: 3.8281 - val_accuracy: 0.2052
Epoch 6/50
2116/2116 [==============================] - 27s 13ms/step - loss: 3.2605 - accuracy: 0.2801 - val_loss: 3.5279 - val_accuracy: 0.2508
Epoch 7/50
2116/2116 [==============================] - 27s 13ms/step - loss: 3.0584 - accuracy: 0.3157 - val_loss: 3.7335 -

In [ ]:
resnet18.save("/content/drive/MyDrive/resnet18.h5")

#2 RESNET

In [ ]:
# arquitetura
input = keras.layers.Input(shape=(img_height, img_width, 3))
rescale = keras.layers.Rescaling(1.0 / 255)(input)
conv_in = keras.layers.Conv2D(
    filters=64, kernel_size=(7, 7), strides=(2, 2), activation="relu"
)(rescale)
pool_in = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding = "same")(conv_in)

conv_r1_1 = keras.layers.Conv2D(64, 3, 1, padding = "same")(pool_in)
bn_r1_1 = keras.layers.BatchNormalization()(conv_r1_1)
relu_r1_1 = keras.layers.ReLU()(bn_r1_1)
conv_r1_2 = keras.layers.Conv2D(64, 3, 1, padding = "same")(relu_r1_1)
bn_r1_2 = keras.layers.BatchNormalization()(conv_r1_2)
skip_r1 = keras.layers.Add()([bn_r1_2, pool_in])
relu_r1_2 = keras.layers.ReLU()(skip_r1)

conv_r2_1 = keras.layers.Conv2D(64, 3, 1, padding = "same")(relu_r1_2)
bn_r2_1 = keras.layers.BatchNormalization()(conv_r2_1)
relu_r2_1 = keras.layers.ReLU()(bn_r2_1)
conv_r2_2 = keras.layers.Conv2D(64, 3, 1, padding = "same")(relu_r2_1)
bn_r2_2 = keras.layers.BatchNormalization()(conv_r2_2)
skip_r2 = keras.layers.Add()([bn_r2_2, relu_r1_2])
relu_r2_2 = keras.layers.ReLU()(skip_r2)

pool_out = keras.layers.GlobalAveragePooling2D()(relu_r2_2)
dense_out = keras.layers.Dense(512, activation="relu")(pool_out)
output = keras.layers.Dense(num_classes, activation="softmax")(dense_out)

resnet18_2 = keras.Model(inputs=input, outputs=output)

# parâmetros de otimização

resnet18_2.compile(optimizer=keras.optimizers.SGD(learning_rate=1), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# parâmetros de execução

early_stopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, start_from_epoch=10)



In [ ]:
resnet18_2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv2d_7 (Conv2D)              (None, 109, 109, 64  9472        ['rescaling_1[0][0]']            
                                )                                                                 
                                                                                            

In [ ]:
r = resnet18_2.fit(train_ds, validation_data=val_ds, epochs = 50, batch_size = batch_size)

# MODELO INCEPTION V3

In [ ]:
# 1. Create a base model with tf.keras.applications

base_model = keras.applications.InceptionV3(include_top= False,)

# 2. Freeze the base model
base_model.trainable = False

#3. Create inputs into models
inputs = keras.layers.Input(shape =(299,299,3), name = "input-layer")

#4. Rescaling
x0 = keras.layers.experimental.preprocessing.Rescaling(1/255.)(inputs)

#5. Pass the inputs
x = base_model(x0)
print(f"Shape after passing inputs through base model: {x.shape}")

# 6. Average pool the outputs of the base model
x2 = keras.layers.GlobalAveragePooling2D(name = "global_average_pooling_layer")(x)
print(f"Shape after GlobalAveragePooling2D: {x.shape}")

#7. Create the output activation layer
dense_out = keras.layers.Dense(1024, activation="relu")(x2)
outputs = keras.layers.Dense(num_classes, activation = "softmax", name = "output-layer")(dense_out)

# 8. Combine the inputs with outputs into a model
inceptionv3 = keras.Model(inputs, outputs)

# 9. Compile the model
inceptionv3.compile(loss = "sparse_categorical_crossentropy",
                optimizer = keras.optimizers.Adam(learning_rate = 0.01),
                metrics = ["accuracy"])

early_stopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, start_from_epoch=10)


87910968/87910968 [==============================] - 5s 0us/step
Shape after passing inputs through base model: (None, 8, 8, 2048)
Shape after GlobalAveragePooling2D: (None, 8, 8, 2048)


In [ ]:
inceptionv3.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input-layer (InputLayer)    [(None, 299, 299, 3)]     0         
                                                                 
 rescaling_2 (Rescaling)     (None, 299, 299, 3)       0         
                                                                 
 inception_v3 (Functional)   (None, None, None, 2048)  21802784  
                                                                 
 global_average_pooling_laye  (None, 2048)             0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 dense_6 (Dense)             (None, 1024)              2098176   
                                                                 
 output-layer (Dense)        (None, 525)               538125    
                                                           

In [ ]:
iv3 = inceptionv3.fit(train_ds,
                      epochs=50,
                      validation_data = val_ds,
                      callbacks = [early_stopping])